In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from isce2_topsapp import download_slcs, download_orbits, download_dem_for_isce2, download_aux_cal

# Input Data

In [ ]:
data = {
        "reference_scenes": ["S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9"],
        "secondary_scenes": ["S1B_IW_SLC__1SDV_20210711T014922_20210711T014949_027740_034F80_859D",
                             "S1B_IW_SLC__1SDV_20210711T014947_20210711T015013_027740_034F80_D404",
                             "S1B_IW_SLC__1SDV_20210711T015011_20210711T015038_027740_034F80_376C"]
        }
reference_scenes = data['reference_scenes']
secondary_scenes = data['secondary_scenes']

In [ ]:
dry_run = True

# SLC

Do everything but download SLCs
*Note, if you encounter `NetrcParseError: access too permissive` error, try to update permissions with `chmod 0600 ~/.netrc`

In [ ]:
out_slc = download_slcs(reference_scenes,
                        secondary_scenes,
                        dry_run=dry_run)
out_slc

# Orbits

In [ ]:
out_orbit = download_orbits(reference_scenes,
                            secondary_scenes,
                            dry_run=dry_run)
out_orbit

# DEM

Don't currently have "dry-run" for this localization.

In [ ]:
out_dem = download_dem_for_isce2(out_slc['extent'])
out_dem

# Aux Cal

In [ ]:
out_aux_cal = download_aux_cal()

# Combining Metadata

In [ ]:
out = {**out_slc,
       **out_dem,
       **out_aux_cal,
       **out_orbit}
out

# Check Name

In [ ]:
from isce2_topsapp.packaging import get_gunw_id

gunw_id = get_gunw_id(reference_properties=out['reference_properties'],
                      secondary_properties=out['secondary_properties'],
                      extent=out['extent'])
gunw_id
